## Equally weighted portfolio of FAANG stocks 

- Facebook (META)
- Apple (AAPL)
- Amazon (AMZN)
- Netflix (NFLX)
- Google (GOOG)

### Library Imports

In [8]:
import numpy as np
import pandas as pd
import yfinance as yf
import xlsxwriter
import math

### Define a new function to retrieve stock prices

In [32]:
def get_stock_price(ticker):
    stock = yf.Ticker(ticker)
    hist = stock.history(period="1d")
    return hist['Close'][0] if not hist.empty else None

### Vector of Stocks Tickers

In [44]:
stocks_input = input('Please provide the stocks ticker (e.g. META, FB, AAPL): ')

stocks = stocks_input.split(',')
stocks = [ticker.strip() for ticker in stocks]

print(' ')
print('You have chosen to invest in the following stocks:', stocks)

Please provide the stocks ticker (e.g. META, FB, AAPL):  META, AAPL, AMZN, NFLX, GOOG


 
You have chosen to invest in the following stocks: ['META', 'AAPL', 'AMZN', 'NFLX', 'GOOG']


### Retrieve Stock Price

In [45]:
my_columns = ['Ticker','Stock Price','Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns=my_columns)

for stock in stocks:
    price = get_stock_price(stock)
    new_row = pd.DataFrame({'Ticker': [stock], 'Stock Price': [price], 'Number of Shares to Buy': ['N/A']})
    final_dataframe = pd.concat([final_dataframe, new_row], ignore_index=True)
    
final_dataframe

,Ticker,Stock Price,Number of Shares to Buy
0,META,471.910004,N/A
1,AAPL,189.869995,N/A
2,AMZN,184.699997,N/A
3,NFLX,621.099976,N/A
4,GOOG,177.289993,N/A


### Compute the Number of Shares to Buy 

In [46]:
portfolio_size = input('Enter the size of your portfolio: ')

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! /n Please try again:")
    portfolio_size = input('Enter the value of your portfolio: ')
    val = float(portfolio_size)

Enter the size of your portfolio:  1000000


In [47]:
position_size = val/len(final_dataframe.index)

for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i, 'Stock Price'])

final_dataframe

,Ticker,Stock Price,Number of Shares to Buy
0,META,471.910004,423
1,AAPL,189.869995,1053
2,AMZN,184.699997,1082
3,NFLX,621.099976,322
4,GOOG,177.289993,1128


### Save the Output as an Excel file 

In [42]:
writer = pd.ExcelWriter('recommended trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index=False)

background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format':'$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format':'0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [43]:
columns_format = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Number of Shares to Buy', integer_format]
}

for column in columns_format.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, columns_format[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', columns_format[column][0], columns_format[column][1])
    
writer.close()